In [1]:
import sqlite3
import speech_recognition as sr
from transformers import pipeline

# Configuración de la base de datos SQLite
DB_NAME = "asistente_hospital.db"


# Función para conectar a la base de datos SQLite
def conectar_db():
    conexion = sqlite3.connect(DB_NAME)
    return conexion


# Función para crear la tabla de consultas y respuestas en la base de datos
def crear_base_datos():
    conexion = conectar_db()
    cursor = conexion.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS consultas_respuestas (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            consulta TEXT,
            respuesta TEXT
        )
    """)
    conexion.commit()
    cursor.close()
    conexion.close()


# Función para insertar una consulta y respuesta en la base de datos
def insertar_consulta_respuesta(consulta, respuesta):
    conexion = conectar_db()
    cursor = conexion.cursor()
    cursor.execute("INSERT INTO consultas_respuestas (consulta, respuesta) VALUES (?, ?)", (consulta, respuesta))
    conexion.commit()
    cursor.close()
    conexion.close()

/opt/anaconda3/envs/machine_learning/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sqlite3
import speech_recognition as sr
import pyttsx3
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
# Configurar reconocimiento de voz
recognizer = sr.Recognizer()

# Configurar síntesis de voz
engine = pyttsx3.init()
engine.setProperty('rate', 150)  # Velocidad de habla
engine.setProperty('volume', 1)  # Volumen

# Configurar el modelo de Transformers
_model_name = "ITG/DialoGPT-medium-spanish-chitchat"
tokenizer = AutoTokenizer.from_pretrained(_model_name)
model = AutoModelForCausalLM.from_pretrained(_model_name)

# Función para generar respuestas
def generate_response(user_input, chat_history_ids=None):
    # Codificar la entrada
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')
    bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1) if chat_history_ids is not None else new_input_ids

    # Generar respuesta con el modelo
    chat_history_ids = model.generate(bot_input_ids,
                                      max_length=150,  # Limitar el tamaño de la respuesta
                                      num_return_sequences=1,  # Solo devolver una respuesta
                                      no_repeat_ngram_size=2,  # Evitar repeticiones en la respuesta
                                      top_p=0.95,  # Controlar la diversidad de la respuesta
                                      top_k=50,  # Controlar la diversidad de la respuesta
                                      pad_token_id=tokenizer.eos_token_id)

    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    return response, chat_history_ids


# Función para escuchar el comando del usuario
def listen_command():
    with sr.Microphone() as source:
        print("Escuchando...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

        try:
            command = recognizer.recognize_google(audio, language='es-ES')
            print("Tú: " + command)
            return command
        except sr.UnknownValueError:
            print("No te entendí. Por favor, repite.")
            return None
        except sr.RequestError:
            print("No se pudo obtener resultados del servicio de Google Speech Recognition")
            return None

# Función para hablar
def speak(text):
    engine.say(text)
    engine.runAndWait()

# Función principal del asistente de voz
def voice_assistant():
    print("¡Hola! Soy tu asistente de voz. (di 'adiós' para terminar la conversación)")
    chat_history_ids = None
    while True:
        command = listen_command()
        if command is None:
            continue
        if command.lower() in ["adiós", "chao", "hasta luego"]:
            speak("¡Hasta luego!")
            break
    
        response, chat_history_ids = generate_response(command, chat_history_ids)
        print("Asistente:", response)
        speak(response)
        
        # Registrar la consulta y la respuesta en la base de datos
        insertar_consulta_respuesta(command, response)

if __name__ == "__main__":
    # Crear la base de datos y la tabla si no existe
    crear_base_datos()

    # Ejecutar el asistente de voz
    voice_assistant()


¡Hola! Soy tu asistente de voz. (di 'adiós' para terminar la conversación)
Escuchando...


/opt/anaconda3/envs/machine_learning/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Tú: hola me das unos consejos para el dolor de
Asistente: ¡Me alegro de que te rías!
Escuchando...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Tú: suponiendo que eres un doctor qué recomendaciones me das para la fiebre
Asistente: !! translated
Escuchando...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Tú: me puedes dar consejos para la fiebre
Asistente: El amor no está entre mis habilidades.
Escuchando...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Tú: eres un médico y quiero que me des consejos saludables
Asistente: Creo haberte hecho una pregunta.! !! hasta mañana! neto! lunar. net! bla blanca! solitario!
Escuchando...


KeyboardInterrupt: 